In [1]:
import subprocess as sp
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import sys, os
import signal
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
from matplotlib import colors
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('png')
plt.figure(figsize=(5,7))
import ipywidgets as widgets

<Figure size 500x700 with 0 Axes>

## Get the abservation x feature matrix

In [14]:
# print(np.where(np.logical_and(contact_min[:,1] == contact1[1,0], contact_min[:,2] == contact1[1,1])))
# print(contact1[1,0], contact1[1,1], contact_min[4679,:])
# w = contact_min[np.where(np.logical_and(contact_min[:,1] == contact1[1,0], contact_min[:,2] == contact1[1,1]))]
# print(w.shape)

In [ ]:
## Input list
dcd = "/full/path/to/*.dcd" 
top = "/full/path/to/*.prmtop" 
pwd = "/full/path/to/working/direcory" # location contactins contact map, output of Contact_book_PI notebook and will be output location
cutoff = 100 # take a large curoff because we want to record all distances for selected pairs
tcl_loc = "/full/path/to/tcl/files" # location of tcl files
number_of_frames = 14900 # frame number in the trajectory
outfile = "tgRNA_HEPN2_dist_feature_1" # outfile prefix

In [8]:
# Load the contact map (say, a 10A cutoff and contacts selected from SYS_A), i.e. the contact/features we will be using to get the observation*feature matrix
contact = np.loadtxt(os.path.join(pwd, "crRNA_HEPN2_10A_new")+".dat")
print(contact.shape)

# Load the distance records of all contacts (in SYS_A/SYS_B) within a chosen cut-off(say 10A/20A).
# This will save some time because, otherwise whatever contacts loaded in 'contact' array,
# distance has to be calculated for all of them over the whole trajectory. 
# Instead, we will just fillup the 'feature_mat' array with records from 'contact_min' array,
# and whatever missing by frames/ by residues, we will calculate using VMD.
contact_min = np.loadtxt(os.path.join(pwd, "contact_all_tgRNA_HEPN2_20A_sample")+".dat")
print(contact_min.shape)

(300, 3)


 contact array description:
 <br> contact = ['resid' 'resid' 'freq']
<br> contact_min = ['frame' 'resid' 'resid' 'distance']

In [ ]:
import time

def contact_distance(contact_min, contact, frame_number, top, dcd, cutoff, pwd, outfile):
    start = time.time()
    feature_mat =  np.zeros(shape=(number_of_frames,contact.shape[0]))
    print("shape of feature matrix:", feature_mat.shape)
    for j in range(feature_mat.shape[1]):
        
# Let's finds out what frames has this contact and then extract distance information from those frames and store that in 'w'
# shape of w = number_of_frames_we_already_found_the_contact, number of column in contact_min (i.e. 4)
# Then we check whether the number of frames in 'w' and 'total number of frames' same or not
# Based on this check, we conditionally fill the feature matrix
        w = contact_min[np.logical_and(contact_min[:,1] == contact1[j,0], contact_min[:,2] == contact1[j,1])]
        
        if (feature_mat.shape[0] - w.shape[0]) == 0:
            feature_mat[:,j] = w[:,3]
            
        elif (feature_mat.shape[0] - w.shape[0]) < (feature_mat.shape[0]*0.01): ## less than 0.01 % of the total frames
            a = w[:,0].tolist()
            b = np.arange(1, feature_mat.shape[0]+1)
            c = []
            if b[-1] != a[-1]:
                a.append(0)
            for i, k in enumerate(b):
                if k != int(a[i]):
                    c.append(k)
                    a.insert(i, 0)
            for i in c:
                row_n = i-1 ##last row
                if i > 11:
                    w = np.insert(w,row_n,w[row_n-1,:],axis=0)
                    w[row_n,3] = np.sum(w[row_n-10:row_n, 3])/10
                    w[row_n,0] = row_n+1
                else:
                    w = np.insert(w,row_n,w[row_n+1,:],axis=0)
                    w[row_n,3] = np.sum(w[row_n+1:row_n+11, 3])/10
                    w[row_n,0] = row_n+1
            if (feature_mat.shape[0] - w.shape[0]) == 0:
                feature_mat[:,j] = w[:,3]

# If the difference between the number of frames in 'w' and 'total number of frames' is within a given range,
# calculate distance for selected frames only. Decide the range based on your trajectory.
# I generally choose the range 0.01% to 90% 
        elif (((feature_mat.shape[0] - w.shape[0]) > (feature_mat.shape[0]*0.01)) & ((feature_mat.shape[0] - w.shape[0]) < (feature_mat.shape[0]*0.90))):
            print((feature_mat.shape[0] - w.shape[0]), "..listed frames")
            a = w[:,0].tolist()
            b = np.arange(1, feature_mat.shape[0]+1)
            c = []
            if b[-1] != a[-1]:
                a.append(0)
            for i, k in enumerate(b):
                if k != int(a[i]):
                    c.append(k)
                    a.insert(i, 0)
            strlist = ' '.join(str(e) for e in c)
            strnew = "[list "+strlist+"]"
            sed = 'sed '+'\"s/top_file/'+top+'/g; s/dcd_file/'+dcd+'/g; s/cutoff_dist/'+str(cutoff)+'/g; s/selection1/'+'resid '+str(int(contact1[j,0]))+'/g; s/selection2/'+'resid '+str(int(contact1[j,1]))+'/g\" '+'vmd_getDist_selectedFrame.tcl >'+' temp.tcl'
            sp.run(sed, shell=True)
            sed = 'sed '+'\"s/fm/'+strnew+'/g\" '+'temp.tcl >'+' contact_getDist_selectedFrame.tcl'
            sp.run(sed, shell=True)
            if os.path.exists("contact_all_temp2.dat"):
                os.remove("contact_all_temp2.dat")
            os.system('/software/repo/moleculardynamics/vmd/1.9.3/vmd -dispdev text -eofexit < contact_getDist_selectedFrame.tcl > output2.log')
            missing_contact = np.loadtxt(os.path.join(pwd,"contact_all_temp2.dat"))
            for l, i in enumerate(c):
                row_n = i-1 ##last row
                if i > 11:
                    w = np.insert(w,row_n,w[row_n-1,:],axis=0)
                else:
                    w = np.insert(w,row_n,w[row_n+1,:],axis=0)
                w[row_n,0] = i
                if i == int(missing_contact[l,0]):
                    w[row_n,3] = missing_contact[l,1]
            if (feature_mat.shape[0] - w.shape[0]) == 0:
                feature_mat[:,j] = w[:,3]
            
# If the difference between the number of frames in 'w' and 'total number of frames' is more than 90% (or your choice)
# calculate distance for all frames in the trajectory.
        else:
            print((feature_mat.shape[0] - w.shape[0]), "..whole dcd")
            sed = 'sed '+'\"s/top_file/'+top+'/g; s/dcd_file/'+dcd+'/g; s/cutoff_dist/'+str(cutoff)+'/g; s/selection1/'+'resid '+str(int(contact1[j,0]))+'/g; s/selection2/'+'resid '+str(int(contact1[j,1]))+'/g\" '+'vmd_getDist_allframes.tcl >'+' contact_getDist_allframes.tcl'
            sp.run(sed, shell=True)
            if os.path.exists("contact_all_temp.dat"):
                os.remove("contact_all_temp.dat")
            os.system('/software/repo/moleculardynamics/vmd/1.9.3/vmd -dispdev text -eofexit < contact_final_selectedContact.tcl > output.log')
            os.system('awk \'{if ($1<14901) print $0}\' contact_all_temp.dat > temp.dat')
            missing_contact = np.loadtxt(os.path.join(pwd,"temp.dat"))
            feature_mat[:,j] = missing_contact[:,1]
    
    print(feature_mat.shape)
    np.save(os.path.join(pwd, outfile)+".npy", feature_mat)
    end = time.time()
    print((end - start)/60, "mins")


In [ ]:
# Usage : For all contacts in the contact map

contact_distance(contact_min, contact, number_of_frames, top, dcd, cutoff, pwd, outfile)

# Usage : For a subset of contacts in the contact map.
contact1 = contact[:300,] # say only top 300 selected contacts
contact_distance(contact_min, contact1, number_of_frames, top, dcd, cutoff, pwd, outfile)